In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install datasets

In [3]:
!pip install accelerate --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [4]:
import pandas as pd
import numpy as np
import random
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [ ]:
!pwd

/content


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
category = pd.read_csv('/content/drive/MyDrive/category.csv')
category.head(20)

,category,label
0,Баталгаажуулах код ирээгүй,0
1,Буцаалтын хүсэлт,1
2,Бүтээгдэхүүн ирээгүй,2
3,Бүтээгдэхүүн солих,3
4,Давхар төлбөр,4
5,Захиалга цуцлах,5
6,Захиалгын мэдээлэл буруу бүртгэгдсэн,6
7,Захиалгын мэдээлэл лавлах,7
8,Захиалгын төлөв,8
9,Захиалгын төлөв шалгах,9


In [ ]:
category = pd.DataFrame(category)
labels_to_drop = ["бүтээгдэхүүн солих", "захиалгын мэдээлэл буруу бүртгэгдсэн"]

category = category[~category['category'].isin(labels_to_drop)]
category

,category,label
0,Баталгаажуулах код ирээгүй,0
1,Буцаалтын хүсэлт,1
2,Бүтээгдэхүүн ирээгүй,2
3,Бүтээгдэхүүн солих,3
4,Давхар төлбөр,4
5,Захиалга цуцлах,5
6,Захиалгын мэдээлэл буруу бүртгэгдсэн,6
7,Захиалгын мэдээлэл лавлах,7
8,Захиалгын төлөв,8
9,Захиалгын төлөв шалгах,9


In [7]:
df = pd.read_csv('/content/drive/MyDrive/train.csv')
df.head()

,text_id,label,text
0,0,Буцаалтын хүсэлт,31 цагт [ЗАХИАЛГЫН ДУГААР] захиалгын [БҮТЭЭГДЭ...
1,1,Захиалгын мэдээлэл лавлах,Сайн байна уу [ХЭРЭГЛЭГЧ] нэртэй хэрэглэгчийн ...
2,2,Бүтээгдэхүүн ирээгүй,"Сайн байна уу, [ЗАХИАЛГЫН ДУГААР] захиалгын ду..."
3,3,захиалгын мэдээлэл буруу бүртгэгдсэн,Хүргэлтийн код ирдэг имэйл хаяг буруу бүртгэгд...
4,4,Бүтээгдэхүүн солих,Сувд нь онлайн дэлгүүрээр [БАРААНЫ ДУГААР] дуг...


In [8]:
df['text'] = df['text'].str.lower()

In [ ]:
len(df)

8322

In [ ]:
def capitalize_first_letter(s):
    return s.capitalize()
df['label'] = df['label'].apply(capitalize_first_letter)
df.head(5)

,text_id,label,text
0,0,Буцаалтын хүсэлт,31 цагт [захиалгын дугаар] захиалгын [бүтээгдэ...
1,1,Захиалгын мэдээлэл лавлах,сайн байна уу [хэрэглэгч] нэртэй хэрэглэгчийн ...
2,2,Бүтээгдэхүүн ирээгүй,"сайн байна уу, [захиалгын дугаар] захиалгын ду..."
3,3,Захиалгын мэдээлэл буруу бүртгэгдсэн,хүргэлтийн код ирдэг имэйл хаяг буруу бүртгэгд...
4,4,Бүтээгдэхүүн солих,сувд нь онлайн дэлгүүрээр [барааны дугаар] дуг...


In [ ]:
# 0 Баталгаажуулах код ирээгүй - 294
# 1 Буцаалтын хүсэлт - 649
# 2 Бүтээгдэхүүн ирээгүй - 2680
# 3	Бүтээгдэхүүн солих - 296
# 4	Давхар төлбөр	- 797
# 5	Захиалга цуцлах	- 408
# 6	Захиалгын мэдээлэл буруу бүртгэгдсэн -	693
# 7	Захиалгын мэдээлэл лавлах - 458
# 8	Захиалгын төлөв	- 553
# 9	Захиалгын төлөв шалгах - 766
# 10	Захиалсан барааны тоо хэмжээ буруу	- 130
# 11	Худалдан авалтын алдаа	- 812

In [ ]:
df.head()

,text_id,label,text
0,0,Буцаалтын хүсэлт,31 цагт [захиалгын дугаар] захиалгын [бүтээгдэ...
1,1,Захиалгын мэдээлэл лавлах,сайн байна уу [хэрэглэгч] нэртэй хэрэглэгчийн ...
2,2,Бүтээгдэхүүн ирээгүй,"сайн байна уу, [захиалгын дугаар] захиалгын ду..."
3,3,Захиалгын мэдээлэл буруу бүртгэгдсэн,хүргэлтийн код ирдэг имэйл хаяг буруу бүртгэгд...
4,4,Бүтээгдэхүүн солих,сувд нь онлайн дэлгүүрээр [барааны дугаар] дуг...


In [9]:
grouped = df.groupby(['label'])['text'].count().reset_index()
grouped.sort_values('text', ascending=False)['text']*100/len(df)


2     31.747176
11     9.757270
4      9.408796
9      9.144436
1      7.690459
7      5.503485
6      5.203076
8      5.167027
5      4.638308
0      3.508772
13     3.124249
3      2.138909
10     1.562124
12     1.405912
Name: text, dtype: float64

In [10]:
test_df = pd.read_csv('/content/drive/MyDrive/test.csv')

In [11]:
test_df['text'] = test_df['text'].str.lower()

In [ ]:
test_df.head()

,text_id,text
0,0,худалдан авагч нь [захиалгын дугаар] захиалгаа...
1,1,гадаадад сурч ажиллаж байгаа [тоог нууцлах] мя...
2,2,"маркетын вэбсайтаас нийт 20,000 төгрөгийн хата..."
3,3,[бүтээгдэхүүний код] кодтой бүтээгдэхүүн [захи...
4,4,[утасны дугаар] [хэрэглэгчийн дугаар] [имэйл] ...


In [12]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

In [13]:
label_encoder.classes_

array(['Баталгаажуулах код ирээгүй', 'Буцаалтын хүсэлт',
       'Бүтээгдэхүүн ирээгүй', 'Бүтээгдэхүүн солих', 'Давхар төлбөр',
       'Захиалга цуцлах', 'Захиалгын мэдээлэл буруу бүртгэгдсэн',
       'Захиалгын мэдээлэл лавлах', 'Захиалгын төлөв',
       'Захиалгын төлөв шалгах', 'Захиалсан барааны тоо хэмжээ буруу',
       'Худалдан авалтын алдаа', 'бүтээгдэхүүн солих',
       'захиалгын мэдээлэл буруу бүртгэгдсэн'], dtype=object)

In [28]:
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

In [16]:
tokenizer = AutoTokenizer.from_pretrained('tugstugi/bert-base-mongolian-uncased', use_fast=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/45.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.01M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [17]:
model = AutoModelForSequenceClassification.from_pretrained('tugstugi/bert-base-mongolian-uncased', num_labels=len(category))

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tugstugi/bert-base-mongolian-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, max_length=128, padding='max_length')

In [18]:
len(tokenizer)

32001

In [20]:
tokenizer.add_tokens(["[захиалгын дугаар]", "[бүтээгдэхүүний код]", "[хүргэлтийн хаяг]", "[бүтээгдэхүүний үнэ]", "[утасны дугаар]", "[барааны дугаар]", "[дэлгүүр нэрс]", "[харилцагч нэрс]", "нууцлагдсан мэдээлэл", "хэрэглэгчийн нэр", "төлбөрийн хэмжээ", "утасны дугаар нууцлагдсан", "худалдааны сайт", "хүргэлтийн хаяг", "бараааны тоо хэмжээ", "тоо хэмжээ", "нууц мэдээлэл", "хүргэлтийн байгууллага", "хүргэлтийн компани", "барааны лугаар", "бүтээгдэхүүний дугаар"])

6

In [21]:
len(tokenizer)

32020

In [22]:
train_df.head()

,text_id,label,text
7514,7514,7,myshop нэвтрэх нэртэй онлайн дэлгүүр ашигладаг...
5220,5220,11,хэрэглэгч онлайн дэлгүүрээс бараа захиалах гэх...
4819,4819,2,хэрэглэгч вэбсайт дээр шинэ [бараа] [хүргэлт] ...
1210,1210,1,[хэрэглэгч]-ийн захиалсан бараа [барааны дугаа...
730,730,7,интернэт дэлгүүрээс захиалгын нэхэмжлэлийг pdf...


In [29]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [24]:
train_dataset[0]

{'text_id': 7514,
 'label': 7,
 'text': 'myshop нэвтрэх нэртэй онлайн дэлгүүр ашигладаг бөгөөд захиалгын түүх excel файл татаж болохгүй харж болохгүй байгаа тул шалгаж өгнө үү. [захиалгын дугаар] захиалгын [огноо] сараас [огноо] хүртэл 1, 1 сараар татсан ч гарахгүй байна.',
 '__index_level_0__': 7514}

In [30]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/7489 [00:00<?, ? examples/s]

Map:   0%|          | 0/833 [00:00<?, ? examples/s]

Map:   0%|          | 0/4100 [00:00<?, ? examples/s]

In [31]:
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch',   columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch',  columns=['input_ids', 'attention_mask'])

In [32]:
print(train_dataset[0])
print(test_dataset[0])

{'label': tensor(2), 'input_ids': tensor([    4,    12,     0,  4236,  9072,  4876,   121,     0,    87,  4637,
         1427,   292,   527, 17210,  5568,   128,  1674,  1526, 12826,   332,
            7,    68,   421,  1008,  1230,   232,     7,     5, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 3

In [33]:
model.bert.embeddings.word_embeddings

Embedding(32000, 768, padding_idx=0)

In [34]:
model.resize_token_embeddings(32021)

Embedding(32021, 768)

In [35]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    eval_strategy="epoch",
    report_to="none"
)

In [36]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [37]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.128003
2,1.463500,1.020868
3,0.810100,1.032097


TrainOutput(global_step=1407, training_loss=0.9497150347939433, metrics={'train_runtime': 524.9638, 'train_samples_per_second': 42.797, 'train_steps_per_second': 2.68, 'total_flos': 1477988246158848.0, 'train_loss': 0.9497150347939433, 'epoch': 3.0})

In [38]:
def classify_sentence(sentence, model, tokenizer, device):
    inputs = tokenizer(sentence, return_tensors='pt', padding='max_length', truncation=True, max_length=128).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions.item()

In [39]:
device = torch.device('cuda')

In [40]:
# Apply the classification function to each sentence in the test set
test_df['label'] = test_df['text'].apply(lambda x: classify_sentence(x, model, tokenizer, device))

In [41]:
test_df.head(20)

,text_id,text,label
0,0,худалдан авагч нь [захиалгын дугаар] захиалгаа...,10
1,1,гадаадад сурч ажиллаж байгаа [тоог нууцлах] мя...,2
2,2,"маркетын вэбсайтаас нийт 20,000 төгрөгийн хата...",11
3,3,[бүтээгдэхүүний код] кодтой бүтээгдэхүүн [захи...,4
4,4,[утасны дугаар] [хэрэглэгчийн дугаар] [имэйл] ...,2
5,5,хэрэглэгч [огноо]-ний өдөр [барааны нэр] бараа...,13
6,6,апп-раа [хугацаа] захиалгын хүргэлт хийгдээгүй...,1
7,7,[бүтээгдэхүүний дугаар] [захиалгын дугаар] [үн...,4
8,8,[огноо]-ны өдөр хэрэглэгч [нууцлах мэдээлэл] х...,2
9,9,онлайн дэлгүүр нь бараа хүргэсэн тул худалдан ...,2


In [42]:
def id2label(id):
    return label_encoder.classes_[id]

In [43]:
test_df['label'] = test_df['label'].apply(id2label)

In [44]:
test_df[['text_id','label']].to_csv('new_token.csv', index=None)

In [45]:
import os
from IPython.display import FileLink

In [ ]:
os.chdir(r'/kaggle/working')

In [ ]:
FileLink(r'new_token.csv')

/content/submission.csv

In [ ]:
#done!